In [1]:
import pandas as pd
import numpy as np

from datetime import datetime 

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, 
                             LearningRateScheduler, TensorBoard)
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


In [2]:
dat = pd.read_csv('../Data/dat_features.csv')

dat = dat.loc[:, "Access":]

C:\Users\Alex\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (2,18) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
unseen = dat[(dat.Impressions.eq(0.0)) & (dat.GRP.eq(0.0)) | (dat.active_flag.eq(1))]
training = dat[dat.Impressions > 0.0]

In [4]:
dat.columns

Index(['Access', 'DMA', 'Date_Aired', 'Estimate', 'GRP', 'Impressions',
       'Length', 'Market', 'Media', 'Spot_Cost', 'Spot_Type', 'Station_ID',
       'Time_Aired', 'DOW', 'daypart', 'Hours', 'Month', 'Quarter', 'creative',
       'is_outlier', 'Station_ID_', 'bins', 'bin_0', 'bin_1', 'bin_2', 'bin_3',
       'bin_4', 'bin_5', 'BP', 'DC', 'DE', 'DP', 'GD', 'GX', 'PL', 'PM', 'PN',
       'PT', 'SR', 'SV', 'TN', 'VE', 'Q119', 'Q219', 'Q319', 'Q419', 'Cable',
       'DirecTV', 'Dish_Network', 'National_Network', 'Over-the-top_content',
       'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday', 'Q1', 'Q2', 'Q3', 'Q4', 'Daytime', 'Early_Fringe',
       'Late_Fringe', 'Late_Night', 'Morning', 'Overnight', 'Primetime',
       'midnight', 'one_am', 'two_am', 'three_am', 'four_am', 'five_am',
       'six_am', 'seven_am', 'eight_am', 'nine_am', 'ten_am', 'eleven_am',
       'noon', 'one_pm', 'two_pm', 'three_pm', 'four_pm', 'five_pm', 'six_pm',
       'seven_

In [5]:
labels = [
        'Q119', 'Q219',
        'Q319', 'Q419',
        'BP', 'DC', 'DE', 'DP',
        'GD', 'GX', 'PL', 'PM', 
        'PN', 'PT', 'SR', 'SV', 
        'TN', 'VE',
        "Length",
        "Spot_Cost",
        "Cable", 
        "DirecTV",
        "Dish_Network",
        "National_Network",
        "Over-the-top_content",
        'Q1', 'Q2', 'Q3', 'Q4',
        'bin_1', 'bin_2',                
        'bin_3', 'bin_4', 'bin_5',
        'Daytime', 'Early_Fringe',
        'Late_Fringe', 'Late_Night', 
        'Morning', 'Overnight', 
        'Primetime'
    ]

X = training.loc[:, labels]
y = training.loc[:, "Impressions"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [6]:
dim = len(X_train.columns)

In [7]:
filepath = '../nn_files/weights.hdf5'
logspath = '../nn_files/logs/scalars/' + datetime.now().strftime("%Y%m%d-%H%M%S")

def scheduler(epoch):
    if epoch < 10:
        return 0.001
    else:
        return float(0.001 * tf.math.exp(0.1*(10-epoch)))
    
callbacks = [TensorBoard(log_dir=logspath),
             EarlyStopping(monitor='val_mean_squared_error', patience=15, mode='min'),
             ModelCheckpoint(filepath, monitor='val_mean_squared_error', verbose=1, mode='min', period=1, save_best_only=True),
             ReduceLROnPlateau(monitor='val_mean_squared_error', patience=2, verbose=1, factor=0.20, min_lr=0.000000000001, mode='min')]


model = Sequential()
model.add(Dense(1024, input_dim=dim, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1))

ad = Adam(lr = 0.00001, 
         amsgrad = True)
model.compile(loss='mean_squared_error', 
              optimizer=ad,
              metrics=['mean_squared_error'])

model.fit(X_train, y_train, 
          epochs = 100, validation_split=0.2, callbacks=callbacks)

Train on 44510 samples, validate on 11128 samples
Epoch 1/100
44510/44510 [==============================] - 12s 264us/step - loss: 1573.3895 - mean_squared_error: 1573.3892 - val_loss: 1441.8285 - val_mean_squared_error: 1441.8290

Epoch 00001: val_mean_squared_error improved from inf to 1441.82898, saving model to ../nn_files/weights.hdf5
Epoch 2/100
29856/44510 [===================>..........] - ETA: 3s - loss: 1557.1016 - mean_squared_error: 1557.1028 

KeyboardInterrupt: 

In [ ]:
preds = model.predict(X_test)

In [35]:
print(np.sqrt(MSE(y_test, preds)))

47.09875159212858
